<a href="https://colab.research.google.com/github/Ar1n382/gmt-projects-/blob/main/GMT225_Assigment2_Ar%C4%B1n_%C3%87ilingir_2230674038.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import math

def xyz2blh(x, y, z, ell):
    """
    Converts Cartesian coordinates (x, y, z) to Ellipsoidal coordinates (phi, lambda, h).

    Inputs:
        x, y, z: Cartesian coordinates in meters [cite: 19]
        ell: Selected ellipsoid (1: GRS80, 2: WGS84, 3: PZ-90.02) [cite: 20]

    Outputs:
        b: Latitude (phi) in degrees
        l: Longitude (lambda) in degrees
        h: Ellipsoidal height in meters
    """

    # 1. Define Ellipsoid Parameters [cite: 27]
    if ell == 1: # GRS 80
        a = 6378137.0
        inv_f = 298.257222101
    elif ell == 2: # WGS 84
        a = 6378137.0
        inv_f = 298.257223563
    elif ell == 3: # PZ-90.02
        a = 6378136.0
        inv_f = 298.257839303
    else:
        raise ValueError("Invalid ellipsoid selection. Choose 1, 2, or 3.")

    # 2. Calculate derived constants
    f = 1 / inv_f
    e2 = 2 * f - f**2  # Square of eccentricity

    # Pre-calculate sqrt(X^2 + Y^2) as it is used frequently
    s = math.sqrt(x**2 + y**2)

    # 3. Calculate Longitude (lambda)
    # Uses atan2 to handle quadrants correctly
    l_rad = math.atan2(y, x)

    # 4. Initial Approximation for Latitude (phi_0) [cite: 11]
    # Note: math functions use radians
    phi_current = math.atan((z / s) / (1 - e2))

    # 5. Iteration Loop [cite: 8, 9]
    # Precision threshold: 10^-8 degrees converted to radians for comparison
    precision_deg = 10**-8
    precision_rad = math.radians(precision_deg)

    while True:
        # Store the phi from the previous step to check convergence later
        phi_prev = phi_current

        # Calculate Radius of Curvature in the prime vertical (N_bar) [cite: 13]
        sin_phi = math.sin(phi_prev)
        N_bar = a / math.sqrt(1 - e2 * sin_phi**2)

        # Calculate Height (h) [cite: 14]
        h = (s / math.cos(phi_prev)) - N_bar

        # Calculate New Latitude (phi_k) [cite: 15]
        # Note: Assuming N_k in the denominator is the same as N_bar
        numerator = z / s
        denominator = 1 - e2 * (N_bar / (N_bar + h))
        phi_current = math.atan(numerator / denominator)

        # Check Convergence
        if abs(phi_current - phi_prev) < precision_rad:
            break

    # 6. Convert outputs to degrees
    b = math.degrees(phi_current)
    l = math.degrees(l_rad)

    return b, l, h

# Example Usage to verify against the PDF example:
if __name__ == "__main__":
    b, l, h = xyz2blh(4210520.621, 1128205.600, 4643227.496, 1)
    print(f"Latitude: {b}") # Should be approx 47.0
    print(f"Longitude: {l}") # Should be approx 15.0
    print(f"Height: {h}")    # Should be approx 2000.0

Latitude: 46.99999999946195
Longitude: 14.999999998583872
Height: 1999.9999960260466
